In [1]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import math
from collections import defaultdict
import csv
import pandas as pd
import numpy as np
from flask import jsonify
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.externals import joblib
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
data = pd.read_csv('house_prediction/data/output copy.csv')

# Separate out the x_data and y_data.
data_X = data.loc[:, [col for col in data.columns if col in ["latitude","longitude","city","state","postal_code","#bedrooms","#bathrooms","sqft"]]]
data_y = data.loc[:, "price"]
#print data_X

In [3]:
""" about label encoder
#https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn
# Encoding the variable
fit = df.apply(lambda x: d[x.name].fit_transform(x))
# Inverse the encoded
fit.apply(lambda x: d[x.name].inverse_transform(x))
# Using the dictionary to label future data
df.apply(lambda x: d[x.name].transform(x))
"""
encode_list = ['city','state']
d = defaultdict(LabelEncoder)
data_X_encoded = data_X.apply(lambda x: d[x.name].fit_transform(x) if x.name in encode_list else x)
#print data_X_encoded

In [4]:
print d['city']
np.save('city.npy', d['city'].classes_)
np.save('state.npy', d['state'].classes_)
le_city = LabelEncoder()
le_city.classes_ = np.load('city.npy')
le_state = LabelEncoder()
le_state.classes_ = np.load('state.npy')
print le_city.transform(['Atlanta','Brookhaven','College Park','Dunwoody','Marietta','Sandy Springs','Smyrna'])


LabelEncoder()
[0 1 2 3 4 5 6]


In [12]:
#input format: latitude	longitude	city	state	postal_code
#  #bedrooms	#bathrooms	sqft	d1	d2	d3	d4	d5	d6	d7	d8	d9	d10	d11	d12	d13	d14	d15	d16	d17	d18	d19	d20
x_test = [33.913258,-84.451048,le_city.transform(['Atlanta'])[0],
          le_state.transform(['GA'])[0],30339,5,7,8407]


In [6]:
rf_model = RandomForestRegressor(n_estimators=150, criterion='mse',
	max_depth=20, min_samples_split=2, min_samples_leaf=1,
	min_weight_fraction_leaf=0.0, max_features='sqrt', 
	max_leaf_nodes=None, min_impurity_decrease=0.0, 
	min_impurity_split=None, bootstrap=True, oob_score=False, 
	n_jobs=1, random_state=None, verbose=0, warm_start=False)

from sklearn.metrics import fbeta_score, make_scorer
def my_custom_loss_func(ground_truth, predictions):
    diff = -np.abs((ground_truth - predictions)/ground_truth).mean()
    return diff
my_scorer = make_scorer(my_custom_loss_func, greater_is_better=True)
cv_para = ShuffleSplit(n_splits=10, test_size=0.1, random_state=100)
print 'cross validation score of rf_model: '+str(cross_val_score(rf_model, data_X_encoded, data_y,cv=cv_para, scoring=my_scorer))
   


cross validation score of rf_model: [-0.23824588 -0.20844122 -0.23356121 -0.21602981 -0.2285855  -0.22002714
 -0.21376681 -0.22638707 -0.21034281 -0.21976963]


In [11]:
cv_score = np.array([-0.23824588, -0.20844122, -0.23356121, -0.21602981, -0.2285855,  -0.22002714,-0.21376681, -0.22638707, -0.21034281, -0.21976963])
cv_score.mean()

-0.221515708

In [8]:
rf_model.fit(data_X_encoded, data_y)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=20,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [15]:
#store the model
joblib.dump(rf_model, 'rf_model_without_d.pkl')
#to use saved model to predict
rf = joblib.load('rf_model_without_d.pkl')
rf.predict([x_test])

array([896189.86666667])